#### With inspiration from 
https://www.analyticsvidhya.com/blog/2016/02/complete-guide-parameter-tuning-gradient-boosting-gbm-python/ 
#### and 
https://medium.com/@pushkarmandot/https-medium-com-pushkarmandot-what-is-lightgbm-how-to-implement-it-how-to-fine-tune-the-parameters-60347819b7fc

In [1]:
import pandas as pd
import numpy as np

import lightgbm as lgb
from lightgbm import LGBMModel,LGBMClassifier

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn import metrics
from sklearn.model_selection import cross_val_score

import matplotlib.pylab as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 12, 4

from sklearn.model_selection import KFold

In [2]:
# read in full datasets

train = pd.read_csv("dataset/train_values.csv")
train_labels = pd.read_csv("dataset/train_labels.csv")
sub = pd.read_csv("dataset/test_values.csv")

In [10]:
# find categorical columns

categorical = [name for t, name in zip(train.dtypes, list(train)) if t != int]

In [11]:
# remove building id

easy_train = train.drop(["building_id"], axis=1)
easy_sub = sub.drop(["building_id"], axis=1)
train_labels = train_labels.drop(["building_id"], axis=1)

In [18]:
# transform categorical features into appropriate types

for col in easy_train.columns:
    if col in categorical:
        easy_train[col] = easy_train[col].astype("category")
        easy_sub[col] = easy_sub[col].astype("category")

In [19]:
# combine all training data

train_data = pd.concat([easy_train, train_labels], axis=1)
target = "damage_grade"

In [20]:
# set aside test set

X_train, X_test, y_train, y_test = train_test_split(easy_train, train_labels, test_size=0.15, random_state=42)

In [64]:
# creat lgbm classfier

lgb = LGBMClassifier(n_estimators=1500, 
                      silent=False, 
                      random_state=43,
                    categoricalfeature=categorical
                     )

In [65]:
# fit with train data 

lgb.fit(X_train, y_train)

LGBMClassifier(boosting_type='gbdt',
               categoricalfeature=['land_surface_condition', 'foundation_type',
                                   'roof_type', 'ground_floor_type',
                                   'other_floor_type', 'position',
                                   'plan_configuration',
                                   'legal_ownership_status'],
               class_weight=None, colsample_bytree=1.0, importance_type='split',
               learning_rate=0.1, max_depth=-1, min_child_samples=20,
               min_child_weight=0.001, min_split_gain=0.0, n_estimators=1500,
               n_jobs=-1, num_leaves=31, objective=None, random_state=43,
               reg_alpha=0.0, reg_lambda=0.0, silent=False, subsample=1.0,
               subsample_for_bin=200000, subsample_freq=0)

In [66]:
# predict 
pred = lgb.predict(X_test)

In [67]:
metrics.f1_score(pred, y_test, average="micro")

0.7434192013506945

In [68]:
# predict with lgb

lgb_prediction = lgb.predict(easy_sub)

# Prediction with LGBM

In [70]:
submission = pd.read_csv("dataset/submission_format.csv")

In [71]:
submission.head()

,building_id,damage_grade
0,300051,1
1,99355,1
2,890251,1
3,745817,1
4,421793,1


In [72]:
submission["damage_grade"] = lgb_prediction

In [74]:
submission.to_csv("submissions/lgbm_est1500_submission.csv", index=False)